In [28]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import ggplot as gplt
from tools.mean_encoder import *
from sklearn.preprocessing import MinMaxScaler
import time

In [5]:
train_set = pd.read_csv("dataset/train_val/training_set.csv")
test_set = pd.read_csv("dataset/train_val/testing_set.csv")

#airport_me = pd.read_csv("./feature_extraction/features/airport_mean_encoding.csv")
#train_set = pd.merge(train_set, airport_me, on="order_id", how="left")

"""train_ae_feature = pd.read_csv("./feature_extraction/train_ae_feature.csv")
train_set = pd.merge(train_set, train_ae_feature, on="order_id", how="left")"""

#long_weekend_feat = pd.read_csv("./feature_extraction/features/long_weekend_feature_2.csv")
train_set["group_id"] = train_set["group_id"].astype(str)
#train_set = pd.merge(train_set, long_weekend_feat, on="group_id", how="left")

"""same_month_group_ratio = pd.read_csv("./feature_extraction/features/same_month_group_ratio.csv")
train_set = pd.merge(train_set, same_month_group_ratio, on="group_id", how="left")"""

group_price_days = pd.read_csv("./feature_extraction/features/group_days_price.csv")
train_set = pd.merge(train_set, group_price_days, on="group_id", how="left")

y_train = train_set["deal_or_not"]
for table in [train_set, test_set]:
    table.drop(columns=["deal_or_not", "group_id", "order_id"], inplace=True)
    
train_set["accum_count_ratio"] = train_set["accum_order"] / train_set["group_id_count"]
    
train_x, val_x, train_y, val_y = train_test_split(train_set, y_train, random_state=104702016,
                                                  shuffle=True, stratify=y_train)

drop_col = ['source1_source2', 'source1_unit', 'source2_unit',
       'source1_source2_unit', 'order_month', 'order_quarter', 'subline_area',
       'begin_month', 'begin_quarter', 'abroad_airport', 'home_airport',
       'abroad_hour', 'abroad_part_of_day', 'abroad_DoY', 'abroad_DoW',
       'abroad_DoM', 'home_DoW', 'home_DoM', 'home_DoY', 'home_hour',
       'home_part_of_day']
drop_col += [x for x in train_x.columns if "prod_vec" in x]
for table in [train_x, val_x]:
    table.drop(columns=drop_col, inplace=True)

#and "DoY" not in col
cat_feature = []
key_words = ["source", "unit", "sub", "area", "order", "begin", "_airport",
             "abroad", "home"]
for i, col in enumerate(train_x.columns):
    for kw in key_words:
        if kw in col and "duration" not in col and "DoY" not in col and "target" not in col \
         and "accum" not in col and "_z" not in col and i not in cat_feature:
            cat_feature.append(i)

cv = StratifiedKFold(5, shuffle=True, random_state=851206)
mean_enc_col = list(train_x.columns[cat_feature])
train_x = mean_encoder(train_x, train_y, mean_enc_col, "deal_or_not", cv)
val_x = test_set_encoder(train_x, train_y, val_x, mean_enc_col, "deal_or_not")

for table in [train_x, val_x]:
    table.drop(columns=mean_enc_col, inplace=True)

C:\Users\S\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\S\travel_data\tools\mean_encoder.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  input_table[target] = y
C:\Users\S\travel_data\tools\mean_encoder.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  x_val[colname] = means
C:\Users\S\travel_data\tools\me

In [18]:
val_pred = logistic_model.predict_proba(val_x)
roc_auc_score(val_y, val_pred[:, 1])

0.6454948014884749

In [6]:
for table in [train_x, val_x]:
    table.replace([np.inf, -np.inf], np.nan, inplace=True)
    table.fillna(table.mean(), inplace=True)

mms = MinMaxScaler()
train_x = mms.fit_transform(train_x)
val_x = mms.transform(val_x)

In [23]:
logistic_model = LogisticRegression(C=1, class_weight="balanced")
logistic_model.fit(train_x, train_y)

LogisticRegression(C=1, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [24]:
train_pred = logistic_model.predict_proba(train_x)
roc_auc_score(train_y, train_pred[:, 1])

0.6450402069439564

In [25]:
val_pred = logistic_model.predict_proba(val_x)
roc_auc_score(val_y, val_pred[:, 1])

0.6474940389544135

In [ ]:
s = time.clock()
svm_model = SVC(kernel="linear", probability=True, random_state=104702016, class_weight="balanced")
svm_model.fit(train_x, train_y)
e = time.clock()
print("time used:", (e-s)/60, "mins.")